In [ ]:
import numpy as np
import ceo
import scipy.io as spio
import matplotlib.pyplot as plt
%matplotlib inline

# AfzIM for Calibration Matrix

In [ ]:
transformation_path = '../SIMCEO/dos/BendingModes_Interface/InterfaceTransformationMatrixes.mat'
load_args = {'squeeze_me' : True, 'mat_dtype' : True}
TransformationM = spio.loadmat(transformation_path,
                  variable_names = ['HR_Qoa','HR_Qcs','Uoa','Ucs'],
                   **load_args)

Qo = TransformationM['HR_Qoa']
Qc = TransformationM['HR_Qcs']
Uo = TransformationM['Uoa']
Uc = TransformationM['Ucs']

modes = u'bending modes'

# SSBM_Softdrive

In [ ]:
transformation_path = '../database/bendingmodes/SoftDrive_SSBM_transformation_matrices.mat'
load_args = {'squeeze_me' : True, 'mat_dtype' : True}
Q_seg = spio.loadmat(transformation_path,variable_names = ['Q_incell'],
                   **load_args)['Q_incell']
U_seg = spio.loadmat(transformation_path, variable_names = ['U_incell'],
                    **load_args)['U_incell']

modes = u'HR PSI'

print(Q_seg[0].shape)
print(Q_seg[6].shape)

print(U_seg[0].shape)
print(U_seg[6].shape)

Qo = Q_seg[0]
Qc = Q_seg[6]

Uo = U_seg[0]
Uc = U_seg[6]

In [ ]:
"""
center = spio.loadmat('m1s7_orth_base.mat')
print("Center\n",center.keys())
print([center[_].shape for  _ in ['Xsurface','Ysurface','Urn']])
outer = spio.loadmat('m1s1_orth_base.mat')
print("Outer",outer.keys())
print([outer[_].shape for  _ in ['Xsurface','Ysurface','Urn']])
""";

---

In [ ]:
"""
xyc =np.hstack([center[_] for _ in ['Xsurface','Ysurface']])
zc = center['Urn'][:,:-3]
xyo =np.hstack([outer[_] for _ in ['Xsurface','Ysurface']])
zo = outer['Urn'][:,:-3]
zc_ext = np.append(zc,np.zeros((zc.shape[0],zo.shape[1]-zc.shape[1])),1)
zc.shape,zo.shape,zc_ext.shape
""";

"""
outputs
((25794, 151), (27685, 162), (25794, 162))
""";

In [ ]:
xyc = Qc[:,[0,1]]
xyo = Qo[:,[0,1]]

zc = Uc[:,:-3] #:-3 for bending Modes
zo = Uo[:,:-3] #:-3 for bending Modes

zc_ext = np.append(zc,np.zeros((zc.shape[0],zo.shape[1]-zc.shape[1])),1)
zc.shape,zo.shape,zc_ext.shape

In [ ]:
L = 8.5 # meter
N_L = 201 # pixel
O = ceo.Mapping(xy=xyo ,z=zo)(N_L,L)
C = ceo.Mapping(xy=xyc ,z=zc_ext)(N_L,L)

In [ ]:
OC = ceo.mapping.cat(O,C,[0]*6+[1])

In [ ]:
OC.dump('bending modes')

In [ ]:
import numpy as np
import ceo
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
gmt = ceo.GMT_MX(M1_mirror_modes= modes ,M1_N_MODE=27)

In [ ]:
src = ceo.Source('V',rays_box_sampling=48*16+1,rays_box_size=25.5,rays_origin=[0,0,25])
src>>(gmt,)
~gmt

gmt.M1.modes.a[:,0] = 1e-5
gmt.M1.modes.update()
+src

plt.imshow(src.phase.host(units='nm'))
plt.colorbar();